# 分享模型

在 Hugging Face，我们相信公开分享知识和资源，能实现人工智能的普及化，让每个人都能受益。我们鼓励您将您的模型与社区分享，以帮助他人节省时间和精力。

要与社区共享模型，您需要在 huggingface.co 上拥有一个帐户。你还可以加入现有的组织或创建一个新的组织。

在本教程中，你将学习两种在 [Model Hub](https://huggingface.co/models) 上共享训练好的或微调的模型的方法：

- 通过编程将文件推送到 Hub。
- 使用 Web 界面将文件拖放到 Hub。

## 仓库功能